In [1]:
import uuid
from datetime import datetime
import random

import values, helpers
import pandas as pd
from create_sap_table import create_table_leanx as ctl

In [2]:
USR02_json = {}
EBAN_json = {}
T161U_json = {}
EKKO_json = {}
T161T_json = {}
T001_json = {}
T024E_json = {}
CDHDR_json = {}
CDPOS_json = {}
LFA1_json = {}
MARA_json = {}
EKPO_json = {}
NAST_json = {}
EKBE_json = {}
MSEG_json = {}
T156_json = {}

# empty tables to pass transformation
EKET_json = {'default':{'MANDT':values.mandt}}
BSEG_json = {'default':{'MANDT':values.mandt}}
BKPF_json = {'default':{'MANDT':values.mandt}}

In [3]:
for k in values.p2p_users.keys():
    USR02_json[k] = {
        "MANDT": values.mandt,
        "BNAME": k,
        "USTYP": values.p2p_users[k]
    }

for k in values.p2p_vendors.keys():
    LFA1_json[k] = {
        "MANDT": values.mandt,
        "LIFNR": uuid.uuid4(),
        "NAME1": k,
        "CCODE": '01' #FIXME 
    }
for k in values.p2p_materials.keys():
    MARA_json[k]= {
        "MANDT": values.mandt,
        "MATNR": uuid.uuid4(),
        "WERKS": "PLNT" #FIXME
    }

In [4]:
def create_pr_item(eban_args, t161u_args, cdhdr_args, cdpos_args):
    EBAN_json[f'{eban_args["BANFN"]}_{eban_args["BNFPO"]}'] = {
        "MANDT": eban_args["MANDT"],
        "BANFN": eban_args["BANFN"],
        "BNFPO": eban_args["BNFPO"],
        "BADAT": eban_args["BADAT"],
        "ERNAM": eban_args["ERNAM"],
        "ESTKZ": eban_args["ESTKZ"],
        "LOEKZ": eban_args["LOEKZ"],
        "AFNAM": eban_args["AFNAM"],
        "LIFNR": eban_args["LIFNR"],
        "WAERS": eban_args["WAERS"],
        "PREIS": eban_args["PREIS"],
        "PEINH": eban_args["PEINH"],
        "MENGE": eban_args["MENGE"],
        "MEINS": eban_args["MEINS"],
        "STATU": eban_args["STATU"],
        "KONNR": eban_args["KONNR"],
        "KTPNR": eban_args["KTPNR"],
        "MATNR": eban_args["MATNR"],
        "WERKS": eban_args["WERKS"],
        "FRGKZ": eban_args["FRGKZ"],
        "TXZ01": eban_args["TXZ01"],
        "BSTYP": eban_args["BSTYP"],
    }
    T161U_json[f'{"EN"}_{eban_args["FRGKZ"]}'] = {
        "MANDT": eban_args["MANDT"],
        "SPRAS": "EN",
        "FRGKZ": eban_args["FRGKZ"],
        "FKZTX": t161u_args["FKZTX"]
    }

    # EBAN is a line item table so there is one entry for each item and same number of CDPOS and CDHDR entries
    CDHDR_json[f'{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}'] = {
        "MANDANT": eban_args["MANDT"],
        "UDATE": cdhdr_args["UDATE"],
        "UTIME": cdhdr_args["UTIME"],
        "USERNAME": cdhdr_args["USERNAME"],
        "CHANGENR": cdhdr_args["CHANGENR"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
    }
    cdpos_tabkey = f'{eban_args["MANDT"]}{eban_args["BANFN"]}{eban_args["BNFPO"]}'
    CDPOS_json[f'{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}_{cdpos_args["TABNAME"]}_{cdpos_tabkey}_{cdpos_args["FNAME"]}_{cdpos_args["CHNGIND"]}'] = {
        "MANDANT": eban_args["MANDT"],
        "TABKEY": cdpos_tabkey,
        "CHANGENR": cdhdr_args["CHANGENR"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "TABNAME": cdpos_args["TABNAME"],
        "FNAME": cdpos_args["FNAME"],
        "CHNGIND": cdpos_args["CHNGIND"],
    }

def create_purchase_order(ekko_args, t161t_args, t024e_args, t001_args, cdhdr_args, cdpos_args):
    EKKO_json[f'{ekko_args["EBELN"]}'] = {
        "MANDT": ekko_args["MANDT"],
        "EBELN": ekko_args["EBELN"],
        "AEDAT": ekko_args["AEDAT"],
        "ERNAM": ekko_args["ERNAM"],
        "LOEKZ": ekko_args["LOEKZ"],
        "LIFNR": ekko_args["LIFNR"],
        "BUKRS": ekko_args["BUKRS"],
        "WAERS": ekko_args["WAERS"],
        "ZTERM": ekko_args["ZTERM"],
        "FRGSX": ekko_args["FRGSX"],
        "ZBD1T": ekko_args["ZBD1T"],
        "ZBD2T": ekko_args["ZBD2T"],
        "ZBD3T": ekko_args["ZBD3T"],
        "ZBD1P": ekko_args["ZBD1P"],
        "ZBD2P": ekko_args["ZBD2P"],
        "BSART": ekko_args["BSART"],
        "RESWK": ekko_args["RESWK"],
        "FRGGR": ekko_args["FRGGR"],
        "EKORG": ekko_args["EKORG"],
        "FRGKE": ekko_args["FRGKE"],
        "STATU": ekko_args["STATU"],
        "BSTYP": ekko_args["BSTYP"],
        "KONNR": ekko_args["KONNR"],
    }

    T161T_json[f'{"EN"}_{ekko_args["BSTYP"]}'] = {
        "MANDT": ekko_args["MANDT"],
        "SPRAS": "EN",
        "BSART": ekko_args["BSART"],
        "BSTYP": ekko_args["BSTYP"],
        "BATXT": t161t_args["BATXT"],
    }

    T024E_json[f'{ekko_args["EKORG"]}'] = {
        "MANDT": ekko_args["MANDT"],
        "EKORG": ekko_args["EKORG"],
        "EKOTX": t024e_args["EKOTX"],
    }

    T001_json[f'{ekko_args["BUKRS"]}'] = {
        "MANDT": ekko_args["MANDT"],
        "BUKRS": ekko_args["BUKRS"],
        "BUTXT": t001_args["BUTXT"],
    }

    CDHDR_json[f'{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}'] = {
        "MANDANT": ekko_args["MANDT"],
        "UDATE": cdhdr_args["UDATE"],
        "UTIME": cdhdr_args["UTIME"],
        "USERNAME": cdhdr_args["USERNAME"],
        "CHANGENR": cdhdr_args["CHANGENR"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
    }
    cdpos_tabkey = f'{ekko_args["MANDT"]}{ekko_args["EBELN"]}'
    CDPOS_json[f'{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}_{cdpos_args["TABNAME"]}_{cdpos_tabkey}_{cdpos_args["FNAME"]}_{cdpos_args["CHNGIND"]}'] = {
        "MANDANT": ekko_args["MANDT"],
        "TABKEY": cdpos_tabkey,
        "CHANGENR": cdhdr_args["CHANGENR"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "TABNAME": cdpos_args["TABNAME"],
        "FNAME": cdpos_args["FNAME"],
        "CHNGIND": cdpos_args["CHNGIND"],
    }

def create_purchase_order_item(ekko_args, ekpo_args, cdhdr_args, cdpos_args):
    EKPO_json[f'{ekko_args["EBELN"]}_{ekpo_args["EBELP"]}'] = {
        "AEDAT": ekpo_args["AEDAT"],
        "AFNAM": ekko_args["ERNAM"],
        "BPRME": ekpo_args["BPRME"],
        "BSTYP": ekko_args["BSTYP"],
        "BUKRS": ekko_args["BUKRS"],
        "DPDAT": ekpo_args["DPDAT"],
        "EBELN": ekko_args["EBELN"],
        "EBELP": ekpo_args["EBELP"],
        "KONNR": ekko_args["KONNR"],
        "KTPNR": ekpo_args["KTPNR"],
        "LOEKZ": ekko_args["LOEKZ"],
        "MANDT": ekko_args["MANDT"],
        "MATNR": ekpo_args["MATNR"],
        "MEINS": ekpo_args["MEINS"],
        "MENGE": ekpo_args["MENGE"],
        "NETPR": ekpo_args["NETPR"],
        "NETWR": ekpo_args["NETWR"],
        "PEINH": ekpo_args["PEINH"],
        "REPOS": ekpo_args["REPOS"],
        "TXZ01": ekpo_args["TXZ01"],
        "UEBTO": ekpo_args["UEBTO"],
        "WEBRE": ekpo_args["WEBRE"],
        "WEPOS": ekpo_args["WEPOS"],
        "WERKS": ekpo_args["WERKS"],
    }

    CDHDR_json[f'{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}'] = {
        "MANDANT": ekko_args["MANDT"],
        "UDATE": cdhdr_args["UDATE"],
        "UTIME": cdhdr_args["UTIME"],
        "USERNAME": cdhdr_args["USERNAME"],
        "CHANGENR": cdhdr_args["CHANGENR"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
    }
    cdpos_tabkey = f'{ekko_args["MANDT"]}{ekko_args["EBELN"]}{ekpo_args["EBELP"]}'
    CDPOS_json[f'{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}_{cdpos_args["TABNAME"]}_{cdpos_tabkey}_{cdpos_args["FNAME"]}_{cdpos_args["CHNGIND"]}'] = {
        "MANDANT": ekko_args["MANDT"],
        "TABKEY": cdpos_tabkey,
        "CHANGENR": cdhdr_args["CHANGENR"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "TABNAME": cdpos_args["TABNAME"],
        "FNAME": cdpos_args["FNAME"],
        "CHNGIND": cdpos_args["CHNGIND"],
    }

def send_purchase_order(ekko_args, nast_args):
    NAST_json[str(uuid.uuid4())] = { 
        #HACK using uuid as key instead of a combination of 8 fields required 
        # all the 8 required fields are accounted for in the fields below
        "AENDE": nast_args["AENDE"],
        "DATVR": nast_args["DATVR"],
        "ERDAT": nast_args["ERDAT"],
        "ERUHR": nast_args["ERUHR"],
        "KAPPL": nast_args["KAPPL"],
        "KSCHL": nast_args["KSCHL"],
        "MANDT": ekko_args["MANDT"],
        "OBJKY": ekko_args["EBELN"],
        "PARNR": nast_args["PARNR"],
        "PARVW": nast_args["PARVW"],
        "SPRAS": "EN",
        "TCODE": nast_args["TCODE"],
        "UHRVR": nast_args["UHRVR"],
        "USNAM": nast_args["USNAM"],
    }

def post_goods_receipt_item(mseg_args, ekbe_args, t156_args):
    MSEG_json[str(uuid.uuid4())] = {
        "BWART": mseg_args["BWART"],
        "CPUDT_MKPF": mseg_args["CPUDT_MKPF"],
        "CPUTM_MKPF": mseg_args["CPUTM_MKPF"],
        "EBELN": mseg_args["EBELN"],
        "EBELP": mseg_args["EBELP"],
        "ERFME": mseg_args["ERFME"],
        "LBKUM": mseg_args["LBKUM"],
        "LFBJA": mseg_args["LFBJA"],
        "LFBNR": mseg_args["LFBNR"],
        "LGORT": mseg_args["LGORT"],
        "LIFNR": mseg_args["LIFNR"],
        "MANDT": mseg_args["MANDT"],
        "MATNR": mseg_args["MATNR"],
        "MBLNR": mseg_args["MBLNR"],
        "MEINS": mseg_args["MEINS"],
        "MENGE": mseg_args["MENGE"],
        "MJAHR": mseg_args["MJAHR"],
        "SHKZG": mseg_args["SHKZG"],
        "SJAHR": mseg_args["SJAHR"],
        "SMBLN": mseg_args["SMBLN"],
        "SMBLP": mseg_args["SMBLP"],
        "USNAM_MKPF": mseg_args["USNAM_MKPF"],
        "WERKS": mseg_args["WERKS"],
        "ZEILE": mseg_args["ZEILE"],
    }
    EKBE_json[str(uuid.uuid4())] = {
        "BELNR": mseg_args["MBLNR"],
        "BUZEI": mseg_args["ZEILE"],
        "GJAHR": mseg_args["MJAHR"],
        "MANDT": mseg_args["MANDT"],
        "MENGE": mseg_args["MENGE"],
        "VGABE": ekbe_args["VGABE"],
        "WAERS": ekbe_args["WAERS"],
        "WRBTR": ekbe_args["WRBTR"],
        "EBELN": mseg_args["EBELN"],
        "EBELP": ekbe_args["EBELP"],
        "ZEKKN": ekbe_args["ZEKKN"],
    }
    T156_json[str(uuid.uuid4())] = {
        "MANDT": mseg_args["MANDT"],
        "BWART": mseg_args["BWART"],
        "XSTBW": t156_args["XSTBW"],
    }



In [9]:
for po in range(10):
    pr_number = uuid.uuid4()
    pr_req_date = helpers.generate_random_date(start_date=datetime(2022, 1, 1), end_date=datetime(2023, 6, 1))
    latest_date = pr_req_date
    vendor = random.choice(list(values.p2p_vendors.keys()))
    konnr = uuid.uuid4()
    ekko_ebeln = uuid.uuid4()
    materials = list(values.p2p_materials.keys())[:-5]
    quantities = [random.randint(15, 50) for _ in materials]
    processed_by = random.choice(list(values.p2p_users.keys()))
    user = random.choice(list(values.p2p_users.keys()))
    
    # create purchase requisition item
    for i, m in enumerate(materials):
        
        eban_args = {
            "MANDT": values.mandt,
            "BANFN": pr_number,
            "BNFPO": i,
            "BADAT": pr_req_date,
            "ERNAM": processed_by,
            "ESTKZ": 'D', # Direct procurement https://www.leanx.eu/en/sap/table/eban.html
            "LOEKZ": 'F', # FIXME
            "AFNAM": random.choice(list(values.p2p_users_requesters.keys())),
            "LIFNR": vendor,
            "WAERS": "EUR",
            "PREIS": quantities[i]*values.p2p_materials[m]['price'],
            "PEINH": values.p2p_materials[m]['price'],
            "MENGE": quantities[i],
            "MEINS": 'UNT',
            "STATU": 'N', # N for Not edited
            "KONNR": konnr,
            "KTPNR": i,
            "MATNR": MARA_json[m]['MATNR'],
            "WERKS": "PLNT", #FIXME
            "FRGKZ": 'R', # FIXME
            "TXZ01": 'Add text here', # FIXME
            "BSTYP": 'B',
        }

        t161u_args = {
            "FKZTX": 'Add text here', # FIXME
        }

        # changes
        changenr = uuid.uuid4()
        cpri_cdhdr_args = {
            "UDATE": pr_req_date,
            "UTIME": helpers.generate_random_time(),
            "USERNAME": processed_by,
            "CHANGENR": changenr,
            "OBJECTCLAS": 'BANF',
            "OBJECTID": 'EBAN',
        }

        cpri_cdpos_args = {
            "TABNAME": 'EBAN',
            "FNAME": 'KEY',
            "CHNGIND": 'I',
        }

        create_pr_item(eban_args=eban_args, t161u_args=t161u_args, cdhdr_args=cpri_cdhdr_args, cdpos_args=cpri_cdpos_args)

    # crete purchase order
    latest_date += helpers.UPTO_WEEK()
    ekko_args = {
        "MANDT": values.mandt,
        "EBELN": ekko_ebeln,
        "AEDAT": latest_date,
        "ERNAM": user,
        "LOEKZ": 'F', # FIXME
        "LIFNR": vendor,
        "BUKRS": '01', #FIXME
        "WAERS": 'EUR',
        "ZTERM": 'Z030', # FIXME
        "FRGSX": '01', #FIXME
        "KONNR": konnr,
        "ZBD1T": 7,
        "ZBD2T":  14,
        "ZBD3T": 21,
        "ZBD1P": 0.02,
        "ZBD2P": 0.01,
        "BSART": 'DEF', #FIXME
        "RESWK": 'DEF', #FIXME
        "FRGGR": 'DF', #FIXME
        "EKORG": random.choice(list(values.p2p_orgs.keys())),
        "FRGKE": 'D', #FIXME
        "STATU": 'D', #FIXME
        "BSTYP": 'F',
    }
    t161t_args = {
        "BATXT": 'Add text here', # FIXME
    }
    t024e_args = {
        "EKOTX": 'Add text here', # FIXME
    }
    t001_args = {
        "BUTXT": 'Add text here', # FIXME
    }
    cpo_cdhdr_args = {
        "UDATE": latest_date,
        "UTIME": helpers.generate_random_time(),
        "USERNAME": user,
        "CHANGENR": uuid.uuid4(),
        "OBJECTCLAS": 'EINKBELEG',
        "OBJECTID": 'EKKO',
    }
    cpo_cdpos_args = {
        "TABNAME": 'EKKO',
        "FNAME": 'KEY',
        "CHNGIND": 'I',
    }

    create_purchase_order(ekko_args=ekko_args, t161t_args=t161t_args, t024e_args=t024e_args, t001_args=t001_args, cdhdr_args=cpo_cdhdr_args, cdpos_args=cpo_cdpos_args)

    # create purchase order item
    for i, m in enumerate(materials): 
        ekpo_args = {
            "AEDAT": latest_date,
            "BPRME": 'UNT', #FIXME
            "DPDAT": latest_date + helpers.UPTO_MONTH(),
            "EBELP": i,
            "KTPNR": i,
            "MATNR": MARA_json[m]['MATNR'],
            "MEINS": 'UNT',
            "MENGE": quantities[i],
            "NETPR": quantities[i]*values.p2p_materials[m]['price'],
            "NETWR": quantities[i]*values.p2p_materials[m]['price'],
            "PEINH": values.p2p_materials[m]['price'],
            "REPOS": None,
            "TXZ01": 'Add text here', # FIXME
            "UEBTO": 99, #FIXME
            "WEBRE": None,
            "WEPOS": None,
            "WERKS": "PLNT", #FIXME
        }
        cpoi_cdhdr_args = {
        "UDATE": latest_date,
        "UTIME": helpers.generate_random_time(),
        "USERNAME": user,
        "CHANGENR": uuid.uuid4(),
        "OBJECTCLAS": 'EINKBELEG',
        "OBJECTID": 'EKPO',
        }
        cpoi_cdpos_args = {
            "TABNAME": 'EKPO',
            "FNAME": 'KEY',
            "CHNGIND": 'I',
        }

        create_purchase_order_item(ekko_args=ekko_args, ekpo_args=ekpo_args, cdhdr_args=cpoi_cdhdr_args, cdpos_args=cpoi_cdpos_args)

    # send purchase order
    latest_date += helpers.UPTO_WEEK()
    nast_args = {
        "AENDE": None,
        "DATVR": latest_date, #FIXME
        "ERDAT": latest_date,
        "ERUHR": helpers.generate_random_time(),
        "KAPPL": 'EF',
        "KSCHL": 'NEU',
        "PARNR": 'DEF', #FIXME
        "PARVW": 'SH', #FIXME
        "TCODE": 'DEF', #FIXME
        "UHRVR": helpers.generate_random_time(), #FIXME
        "USNAM": user,
    }
    send_purchase_order(ekko_args=ekko_args, nast_args=nast_args)

    # post goods receipt
    latest_date += helpers.UPTO_WEEK()
    for i, m in enumerate(materials): 
        mseg_args = {
            "BWART": '101',
            "CPUDT_MKPF": latest_date,
            "CPUTM_MKPF": helpers.generate_random_time(),
            "EBELN": ekko_ebeln,
            "EBELP": i,
            "ERFME": 'UNT',
            "LBKUM": 99, #FIXME
            "LFBJA": 2022, #FIXME
            "LFBNR": 'DEF', #FIXME
            "LGORT": 'DEF', #FIXME
            "LIFNR": 'DEF', #FIXME,
            "MANDT": values.mandt,
            "MATNR": MARA_json[m]['MATNR'],
            "MBLNR": uuid.uuid4(),
            "MEINS": 'UNT',
            "MENGE": quantities[i],
            "MJAHR": 2022, #FIXME
            "SHKZG": 'S', # Debit
            "SJAHR": 2022, #FIXME
            "SMBLN": 99, #FIXME
            "SMBLP": 99, #FIXME
            "USNAM_MKPF": user,
            "WERKS": 'PLNT', #FIXME,
            "ZEILE": i,
        }
        ekbe_args = {
            "VGABE": '1',
            "WAERS": 'EUR',
            "WRBTR": quantities[i]*values.p2p_materials[m]['price'],
            "EBELP": i,
            "ZEKKN": 99, #FIXME
        }
        t156_args = {
            "XSTBW": None,
        }
        post_goods_receipt_item(mseg_args=mseg_args, ekbe_args=ekbe_args, t156_args=t156_args)
    

In [6]:
USR02_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('USR02')])
USR02 = pd.concat([USR02_full_col, pd.DataFrame(USR02_json.values())])
EKKO_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('EKKO')])
EKKO = pd.concat([EKKO_full_col, pd.DataFrame(EKKO_json.values())])
EBAN_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('EBAN')])
EBAN = pd.concat([EBAN_full_col, pd.DataFrame(EBAN_json.values())])
T161U_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('T161U')])
T161U = pd.concat([T161U_full_col, pd.DataFrame(T161U_json.values())])
T161T_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('T161T')])
T161T = pd.concat([T161T_full_col, pd.DataFrame(T161T_json.values())])
T001_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('T001')])
T001 = pd.concat([T001_full_col, pd.DataFrame(T001_json.values())])
T024E_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('T024E')])
T024E = pd.concat([T024E_full_col, pd.DataFrame(T024E_json.values())])
CDHDR_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('CDHDR')])
CDHDR = pd.concat([CDHDR_full_col, pd.DataFrame(CDHDR_json.values())])
CDPOS_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('CDPOS')])
CDPOS = pd.concat([CDPOS_full_col, pd.DataFrame(CDPOS_json.values())])
LFA1_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('LFA1')])
LFA1 = pd.concat([LFA1_full_col, pd.DataFrame(LFA1_json.values())])
MARA_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('MARA')])
MARA = pd.concat([MARA_full_col, pd.DataFrame(MARA_json.values())])
EKPO_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('EKPO')])
EKPO = pd.concat([EKPO_full_col, pd.DataFrame(EKPO_json.values())])
NAST_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('NAST')])
NAST = pd.concat([NAST_full_col, pd.DataFrame(NAST_json.values())])
EKBE_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('EKBE')])
EKBE = pd.concat([EKBE_full_col, pd.DataFrame(EKBE_json.values())])
MSEG_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('MSEG')])
MSEG = pd.concat([MSEG_full_col, pd.DataFrame(MSEG_json.values())])
T156_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('T156')])
T156 = pd.concat([T156_full_col, pd.DataFrame(T156_json.values())])

# empty tables to pass transformation
EKET_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('EKET')])
EKET = pd.concat([EKET_full_col, pd.DataFrame(EKET_json.values())])
BSEG_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('BSEG')])
BSEG = pd.concat([BSEG_full_col, pd.DataFrame(BSEG_json.values())])
BKPF_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('BKPF')])
BKPF = pd.concat([BKPF_full_col, pd.DataFrame(BKPF_json.values())])

In [7]:
USR02.to_csv('data/P2P/USR02.csv', index=False)
EBAN.to_csv('data/P2P/EBAN.csv', index=False)
T161U.to_csv('data/P2P/T161U.csv', index=False)
EKKO.to_csv('data/P2P/EKKO.csv', index=False)
T161T.to_csv('data/P2P/T161T.csv', index=False)
T001.to_csv('data/P2P/T001.csv', index=False)
T024E.to_csv('data/P2P/T024E.csv', index=False)
CDHDR.to_csv('data/P2P/CDHDR.csv', index=False)
CDPOS.to_csv('data/P2P/CDPOS.csv', index=False)
LFA1.to_csv('data/P2P/LFA1.csv', index=False)
MARA.to_csv('data/P2P/MARA.csv', index=False)
EKPO.to_csv('data/P2P/EKPO.csv', index=False)
NAST.to_csv('data/P2P/NAST.csv', index=False)
EKBE.to_csv('data/P2P/EKBE.csv', index=False)
MSEG.to_csv('data/P2P/MSEG.csv', index=False)
T156.to_csv('data/P2P/T156.csv', index=False)

# empty tables to pass transformation
EKET.to_csv('data/P2P/EKET.csv', index=False)
BSEG.to_csv('data/P2P/BSEG.csv', index=False)
BKPF.to_csv('data/P2P/BKPF.csv', index=False)